In [1]:
#################################################
#created the 23/04/2018 12:57 by Alexis Blanchet#
#################################################


import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import scipy.stats
import matplotlib
import plotly.offline as offline
import plotly.graph_objs as go
import datetime

In [6]:
def get_info(file):
    pd.read_csv(file)
    date = list(file.split('-')[1])
    year = ''.join((date[0],date[1],date[2],date[3]))
    month = ''.join((date[4],date[5]))
    day = ''.join((date[6],date[7]))
    heure = ''.join((date[8],date[9]))
    minutes = ''.join((date[10],date[11]))
    return([year,month,day,heure,minutes])

In [7]:
get_info('odVect.csv-20180417001400')

['2018', '04', '17', '00', '14']

In [58]:
def load_file(file,chaine):
    l = get_info(file)
    df = pd.read_csv(file,sep=';')
    df = df.rename(index=str, columns={"Unnamed: 0": "tf", df.columns.values[1]: "count"})
    print(date)
    df["chaine d'origine"]=df['tf'].apply(get_origin)
    df["chaine d'arrivée"]=df['tf'].apply(get_destination)
    df["date"] = date
    gain = df[df["chaine d'arrivée"]==int(chaine)]['count'].sum()
    loss = df[df["chaine d'origine"]==int(chaine)]['count'].sum()
    temp_df = pd.DataFrame()
    temp_df['année'] = [l[0]]
    temp_df['mois'] = l[1]
    temp_df['jour'] = l[2]
    temp_df['heure'] = l[3]
    temp_df['minute'] = l[4]
    temp_df['loss'] = loss
    temp_df['gain'] = gain
    
    return temp_df


def get_origin(x):
    x = int(x) & ((1<<24)-1)
    return x

def get_destination(x):
    x = int(x)>>24 & ((1<<24)-1)
    return x

In [59]:
df = load_file('odVect.csv-20180417141400',192)

Index(['Unnamed: 0', '1523967240'], dtype='object')
2018-04-17 14:14:00


In [60]:
df

,année,mois,jour,heure,minute,loss,gain
0,2018,04,17,14,14,2613,1816


In [89]:
loss = pd.read_csv('loss.csv').drop(['Unnamed: 0'],axis = 1)
gain = pd.read_csv('gain.csv').drop(['Unnamed: 0'],axis = 1)

In [90]:
loss.describe()

,année,mois,jour,heure,minute,loss
count,11431.0,11431.0,11431.000000,11431.000000,11431.000000,11431.00000
mean,2018.0,4.0,20.116525,11.476861,29.535736,2802.11985
std,0.0,0.0,2.324896,6.945089,17.320597,3632.25416
min,2018.0,4.0,16.000000,0.000000,0.000000,0.00000
25%,2018.0,4.0,18.000000,5.000000,15.000000,836.00000
50%,2018.0,4.0,20.000000,11.000000,30.000000,1776.00000
75%,2018.0,4.0,22.000000,18.000000,45.000000,3458.00000
max,2018.0,4.0,24.000000,23.000000,59.000000,62144.00000


In [95]:
loss = loss[loss['jour']==20]
gain = gain[gain['jour']==20]

In [96]:
loss.head()

,année,mois,jour,heure,minute,loss,time
4652,2018,4,20,0,0,1249,0
9914,2018,4,20,0,1,1520,1
1397,2018,4,20,0,2,1544,2
3304,2018,4,20,0,3,1533,3
8934,2018,4,20,0,4,3603,4


In [97]:
loss['time'] = loss['heure']*60+loss['minute']
gain['time'] = gain['heure']*60+gain['minute']

In [98]:
loss = loss.sort_values('time')
gain = gain.sort_values('time')

In [99]:
loss.head()

,année,mois,jour,heure,minute,loss,time
4652,2018,4,20,0,0,1249,0
9914,2018,4,20,0,1,1520,1
1397,2018,4,20,0,2,1544,2
3304,2018,4,20,0,3,1533,3
8934,2018,4,20,0,4,3603,4


In [100]:
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


data = [go.Bar(
            x=[x/60 for x in loss['time'].values],
            y=loss['loss'].values
    )]
plot(data, filename='loss-bar')

'file:///home/alexis/Bureau/Stage/Vectors/loss-bar.html'

In [101]:
data = [go.Bar(
            x=[x/60 for x in gain['time'].values],
            y=gain['gain'].values
    )]
plot(data, filename='gain-bar')

'file:///home/alexis/Bureau/Stage/Vectors/gain-bar.html'

In [106]:
trace1 = go.Bar(
            x=[x/60 for x in loss['time'].values],
            y=-loss['loss'].values/30000,
    name='loss'
)
trace2 = go.Bar(
            x=[x/60 for x in gain['time'].values],
            y=gain['gain'].values/30000,
    name='gain'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
plot(data, filename='all-bar')

'file:///home/alexis/Bureau/Stage/Vectors/all-bar.html'

In [107]:
l = pd.DataFrame()
l['values'] = loss['loss']


In [109]:
l.to_csv('test.csv',index=False)